In [31]:
import h5py
import numpy as np

In [32]:
data_path = "/data1/jessica/data/toyota/once_data_w_lanes_compressed_train.hfd5" 
people_seqs = []
with h5py.File(data_path, "r") as f:
        for seq_key in list(f.keys()):
            iter_dict = {}
            keys_ = f[seq_key].keys()
            for key in keys_:
                ds_obj = f[seq_key][key][()]
                iter_dict[key] = ds_obj
            people_seqs.append(iter_dict)

In [33]:
angles = [list(seq["angle"]) for seq in people_seqs]

In [34]:
flat_list = np.array([item for sublist in angles for item in sublist])

In [35]:
mi = flat_list.min()
ma = flat_list.max()
rang = ma - mi

In [36]:
mi, ma, rang

(2.1073424e-08, 1.5222348, 1.5222348)

In [37]:
flat_list.sort()

In [39]:
np.median(flat_list)

0.00018448659

In [50]:
an_array = flat_list
mean = np.mean(an_array)
standard_deviation = np.std(an_array)
distance_from_mean = abs(an_array - mean)
max_deviations = 3
not_outlier = distance_from_mean < max_deviations * standard_deviation
no_outliers = an_array[not_outlier]

In [53]:
no_outliers.min(), no_outliers.max(), no_outliers.max()  - no_outliers.min()

(2.1073424e-08, 0.102598816, 0.102598794)